In [32]:
import torch
import torchvision as tv
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from sklearn.model_selection import KFold
import numpy as np
import torch.optim as optim
from tqdm.auto import tqdm
from timeit import default_timer as timer
from copy import deepcopy
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [2]:
try:
   from torchmetrics import ConfusionMatrix
except:
    !pip install torchmetrics
    from torchmetrics import ConfusionMatrix

conmat=ConfusionMatrix(task='multiclass',num_classes=3)
conmat=conmat

In [3]:
# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [4]:
torch.manual_seed(71)
torch.cuda.manual_seed(71)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading data

In [6]:
train_dir = "/content/drive/MyDrive/DataMiningCW/train"
test_dir = "/content/drive/MyDrive/DataMiningCW/test"


In [7]:
data = pd.read_csv('/content/drive/MyDrive/DataMiningCW/train.csv', usecols=['ID', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Total_EMI_per_month', 'Credit_Score'])

In [8]:
data.head()

,ID,Annual_Income,Monthly_Inhand_Salary,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Total_EMI_per_month,Credit_Score
0,5634,19114.12,1824.843333,4.0,3.0,3.0,11.27,4.0,809.98,49.574949,Good
1,5635,19114.12,1824.843333,4.0,3.0,3.0,11.27,4.0,809.98,49.574949,Good
2,5636,19114.12,1824.843333,4.0,3.0,3.0,11.27,4.0,809.98,49.574949,Good
3,5637,19114.12,1824.843333,4.0,3.0,5.0,6.27,4.0,809.98,49.574949,Good
4,5638,19114.12,1824.843333,4.0,3.0,6.0,11.27,4.0,809.98,49.574949,Good


In [9]:
data['Credit_Score'], _ = pd.factorize(data['Credit_Score'])

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     100000 non-null  int64  
 1   Annual_Income          100000 non-null  float64
 2   Monthly_Inhand_Salary  100000 non-null  float64
 3   Num_Credit_Card        100000 non-null  float64
 4   Interest_Rate          100000 non-null  float64
 5   Delay_from_due_date    100000 non-null  float64
 6   Changed_Credit_Limit   100000 non-null  float64
 7   Num_Credit_Inquiries   100000 non-null  float64
 8   Outstanding_Debt       100000 non-null  float64
 9   Total_EMI_per_month    100000 non-null  float64
 10  Credit_Score           100000 non-null  int64  
dtypes: float64(9), int64(2)
memory usage: 8.4 MB


In [11]:
data.isna().sum()

ID                       0
Annual_Income            0
Monthly_Inhand_Salary    0
Num_Credit_Card          0
Interest_Rate            0
Delay_from_due_date      0
Changed_Credit_Limit     0
Num_Credit_Inquiries     0
Outstanding_Debt         0
Total_EMI_per_month      0
Credit_Score             0
dtype: int64

In [12]:
scaler = StandardScaler()
features = data[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Total_EMI_per_month']]  # all columns except the target
target = data['Credit_Score']

# Normalize features
features_normalized = scaler.fit_transform(features)

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features_normalized, target, test_size=0.2, random_state=42)

# Applying Smote

In [14]:
smote = SMOTE(random_state=42)
features_train, target_train = smote.fit_resample(features_train, target_train)

# Preparing data for neural network

In [15]:
train_features = torch.tensor(features_train, dtype = torch.float32)
train_targets = torch.tensor(target_train, dtype = torch.int64).squeeze()

In [18]:
target_test = target_test.to_numpy()

In [19]:
test_features = torch.tensor(features_test, dtype = torch.float32)
test_targets = torch.tensor(target_test, dtype = torch.int64).squeeze()

In [20]:
# Create Tensor datasets
train_dataset = TensorDataset(train_features, train_targets)
test_dataset = TensorDataset(test_features, test_targets)

# Setup DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Useful functions

In [21]:
def train_model(model, train_loader, epochs):
  num_epochs = epochs
  for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.cuda(), labels.cuda()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [22]:
def get_all_predictions(model, data_loader):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

In [23]:
def compute_metrics(predictions, true_labels):
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')
    accuracy = accuracy_score(true_labels, predictions)
    conf_matrix = confusion_matrix(true_labels, predictions)

    return precision, recall, f1, accuracy, conf_matrix

In [24]:

def plot_confusion_matrix(conf_matrix, class_names):
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt="d", linewidths=.5, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

In [25]:
def evaluate_model(model, data_loader):
    predictions, true_labels = get_all_predictions(model, data_loader)

    # Compute metrics
    precision, recall, f1, accuracy, conf_matrix = compute_metrics(predictions, true_labels)

    print(f'Precision: {precision*100}%')
    print(f'Recall: {recall*100}%')
    print(f'F1 Score: {f1*100}%')
    print(f'Accuracy: {accuracy*100}%')
    print("Confusion Matrix:\n", conf_matrix)

# Neural Networks

# Architecture 1: Two convolution layers, one max pooling layer, and two fully connected layers

In [26]:
class SimpleCNN1D(nn.Module):
    def __init__(self):
        super(SimpleCNN1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3, 1)
        self.conv2 = nn.Conv1d(16, 32, 3, 1)
        self.pool = nn.MaxPool1d(2, 2)
        self.fc1 = nn.Linear(32 * 2, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [27]:
model = SimpleCNN1D()
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [28]:
train_model(model, train_loader, 20)

Epoch [1/20], Step [100/1996], Loss: 0.7316
Epoch [1/20], Step [200/1996], Loss: 0.6359
Epoch [1/20], Step [300/1996], Loss: 0.7745
Epoch [1/20], Step [400/1996], Loss: 0.6722
Epoch [1/20], Step [500/1996], Loss: 0.8439
Epoch [1/20], Step [600/1996], Loss: 0.8027
Epoch [1/20], Step [700/1996], Loss: 0.8380
Epoch [1/20], Step [800/1996], Loss: 0.8462
Epoch [1/20], Step [900/1996], Loss: 0.8420
Epoch [1/20], Step [1000/1996], Loss: 0.7729
Epoch [1/20], Step [1100/1996], Loss: 0.6672
Epoch [1/20], Step [1200/1996], Loss: 0.7974
Epoch [1/20], Step [1300/1996], Loss: 0.7765
Epoch [1/20], Step [1400/1996], Loss: 0.7833
Epoch [1/20], Step [1500/1996], Loss: 0.7767
Epoch [1/20], Step [1600/1996], Loss: 0.8202
Epoch [1/20], Step [1700/1996], Loss: 0.7622
Epoch [1/20], Step [1800/1996], Loss: 0.8457
Epoch [1/20], Step [1900/1996], Loss: 0.7922
Epoch [2/20], Step [100/1996], Loss: 0.6841
Epoch [2/20], Step [200/1996], Loss: 0.6763
Epoch [2/20], Step [300/1996], Loss: 0.7490
Epoch [2/20], Step [40

In [35]:
evaluate_model(model, test_loader)

Precision: 69.47624883464538%
Recall: 65.77%
F1 Score: 65.97259774020804%
Accuracy: 65.77%
Confusion Matrix:
 [[2723  663  141]
 [2361 5988 2250]
 [ 482  949 4443]]


In [33]:
class_names = ["Good", "Standard", "Poor"]
plot_confusion_matrix(conf_matrix, class_names)

NameError: name 'conf_matrix' is not defined